Hacker.ipynb
============

This notebook attempts to use my langchain alternatives to hack a target device. The goal of the notebook is to either show the agent working, or to shine a light on what aspects need to be improved. The notebook should be continuously updated to use the latest improvemenets in the agent.


In [1]:
import os
import sys
import time
import openai

from dotenv import load_dotenv

from agent import Agent
from tasker import TaskManager
from entityDB import EntityDatabase
from shelltool import ExecutionSandbox

load_dotenv()
openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# Just ensure that things are properly killed
os.system("podman kill rat-machine-sandbox")
os.system("podman rm rat-machine-sandbox")
time.sleep(1)

Error: no container with name or ID "rat-machine-sandbox" found: no such container
Error: no container with ID or name "rat-machine-sandbox" found: no such container


In [3]:
task_manager = TaskManager(openai)
database = EntityDatabase(openai, "gpt-3.5-turbo")
agent = Agent(tools=[ExecutionSandbox()], verbose=False)

In [4]:
while True:
    task = task_manager.get_next()

    # ran out of tasks
    if task == "":
        break
        
    
    print("--------------context---------------")
    print(database.database)
    print("------------------------------------")
        
    context = ""
    if database.database != "":
        context = database.get_context(task)


    print("--------------context---------------")
    print(f"Context: {context}")
    print(f"Task: {task}")
    print("------------------------------------")

    
    output = agent.run(task, context=context)
    database.update(output)
    task_manager.mark_next_completed()

    print("--------------output----------------")
    print(output)
    print("------------------------------------\n\n")

--------------context---------------

------------------------------------
--------------context---------------
Context: 
Task: Find your current local IP address for ethernet
------------------------------------
--------------output----------------
The current local IP address for ethernet is 192.168.1.179.
------------------------------------


--------------context---------------
192.168.1.179: The machine with the local IP 192.168.1.179 is the current local IP address for ethernet.
------------------------------------
--------------context---------------
Context: 192.168.1.179
Task: Find the IP range of the current network
------------------------------------
--------------output----------------
The IP range of the current network is from 192.168.1.0 to 192.168.1.255.
------------------------------------


--------------context---------------
192.168.1.0 - 192.168.1.255: The IP range of the current network is from 192.168.1.0 to 192.168.1.255.
------------------------------------
-

Exception in thread Thread-4 (_container_thread):
Traceback (most recent call last):
  File "/home/antone/Music/The-Autonomous-R.A.T.-Machine/research/reinvent-the-wheel/shelltool.py", line 106, in _container_thread
    subprocess.run(
  File "/usr/lib/python3.11/subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['podman', 'run', '--cap-add=NET_RAW', '--network', 'host', '--name', 'rat-machine-sandbox', '--rm', 'rat-machine-image']' returned non-zero exit status 137.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/home/antone/Music/The-Autonomous-R.A.T.-Machine/research/reinvent-the-wheel/shelltool.py", line 125, in _container_thread
    assert False, "Exe

Container exited with error Command '['podman', 'run', '--cap-add=NET_RAW', '--network', 'host', '--name', 'rat-machine-sandbox', '--rm', 'rat-machine-image']' returned non-zero exit status 137.
